<a href="https://colab.research.google.com/github/Zahra-Mhdi/Deep-Learning-Exercises/blob/main/Session_4_CNN_Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_cifar = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

train_dataset_cifar = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
test_dataset_cifar = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

train_loader_cifar = DataLoader(train_dataset_cifar, batch_size=64, shuffle=True)
test_loader_cifar = DataLoader(test_dataset_cifar, batch_size=64, shuffle=False)

vgg = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
for p in vgg.parameters():
    p.requires_grad = False
vgg.classifier[6] = nn.Linear(vgg.classifier[6].in_features, 10)
for p in vgg.classifier[6].parameters():
    p.requires_grad = True
model = vgg.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier[6].parameters(), lr=0.001)

num_epochs = 3
for epoch in range(num_epochs):
    total_loss = 0
    correct = 0
    total = 0
    model.train()
    for image, label in train_loader_cifar:
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    print(epoch+1, total_loss/len(train_loader_cifar), 100*correct/total)

correct = 0
total = 0
model.eval()
with torch.no_grad():
    for image, label in test_loader_cifar:
        image, label = image.to(device), label.to(device)
        output = model(image)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
print(100*correct/total)
